# Q1

El primer paso es seleccionar lo datos desde MongoDB, pasarlos a un dataframe, limpiarlos y guardar las seleccion de datos en un archivo json para volver a llevarlos a MongoDB.

In [112]:
# Tu codigo aqui
import pandas as pd 
import numpy as np
from pymongo import MongoClient
from bson.json_util import dumps
from pandas.io.json import json_normalize
import json
import re

In [113]:
client=MongoClient()
db=client.companiesDB

In [114]:
db.companies.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [176]:
# Realizamos una query para extraer las empresas de nuestro interes.
# videojuegos, empresas de desarrollo web, e-commerce, mobile, foto, video e incluso de network hosting

data = db.companies.find({'$and':[{'$or':[{'category_code':'web'},{'category_code':'network_hosting'},
                                 {'category_code':'photo_video'},{'category_code':'ecommerce'},
                                 {'category_code':'mobile'}]},
                                 {'$and':[{'offices':{'$ne':[]}},
                                          {'number_of_employees':{'$gt':0}},
                                          {'total_money_raised':{'$ne':'$0'}},
                                          {'founded_year':{'$gt':0}},
                                          {'offices.latitude':{'$ne':None}},
                                          {'offices.longitude':{'$ne':None}},
                                          {'deadpooled_year':{'$eq':None}}]}]},
                         {'_id':0,'name':1,'founded_year':1,'category_code':1,'number_of_employees':1,'offices':1,'total_money_raised':1})

In [177]:
interesting_companies = pd.DataFrame.from_dict(data).sort_values('founded_year',ascending=True)

In [178]:
def change_to_M(x):
    nums = ''.join(re.findall(r'\d',x))
    if x[0] == '£':
        nums *= 23
    elif x[0] == '€':
        nums *= 21
        
    if x[-1] == 'M':
        return float(nums)
    elif x[-1] == 'k':
        return float(nums)/1000
    else:
        return float(nums)*1000

def clasify_company(x):
    if x.founded_year > 2005 and x.number_of_employees < 300:
        return 'Start-up'
    else:
        return 'Big company'

def office(x):
    field = {"type": "Point","coordinates": [x[0]['longitude'],x[0]['latitude']]}
    return field


interesting_companies['type'] = interesting_companies.apply(clasify_company,axis=1)
interesting_companies['loc_office'] = interesting_companies['offices'].apply(office)
interesting_companies.drop('offices',axis=1,inplace=True)

In [179]:
interesting_companies.head(5)

,category_code,founded_year,name,number_of_employees,total_money_raised,type,loc_office
320,network_hosting,1961,ECI Telecom,3000,$173M,Big company,"{'type': 'Point', 'coordinates': [-80.1912988,..."
3,network_hosting,1984,Cisco,63000,$2.5M,Big company,"{'type': 'Point', 'coordinates': [-121.95377, ..."
399,mobile,1984,Morcom International,15,$100k,Big company,"{'type': 'Point', 'coordinates': [-77.4276702,..."
32,web,1985,AOL,8000,$1B,Big company,"{'type': 'Point', 'coordinates': [-73.9919311,..."
382,network_hosting,1993,Logicworks,75,$7.6M,Big company,"{'type': 'Point', 'coordinates': [-74.0045549,..."


In [180]:
content = []
dict_companies = interesting_companies.transpose().to_dict()
for i in range(len(dict_companies)):
    content.append(dict_companies[i])

In [181]:
interesting_companies.type.value_counts()

Start-up       266
Big company    139
Name: type, dtype: int64

In [182]:
mycol = db["companies2"]
x = mycol.insert_many(content)